In [1]:
from unittest.mock import Mock, patch, call

import pytest
from unittest.mock import MagicMock

from fastlane_bot import Bot, Config
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3,  CarbonV1, BancorV3
from fastlane_bot.events.managers.manager import Manager
Base = None
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
import asyncio
from unittest.mock import AsyncMock
import nest_asyncio
nest_asyncio.apply()
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *

#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)


ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.2 [requirements >= 3.0 is met]


c:\Users\Kveen\PycharmProjects\fastlane-bot\venv_11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nest_asyncio

<module 'nest_asyncio' from 'c:\\Users\\Kveen\\PycharmProjects\\fastlane-bot\\venv_11\\Lib\\site-packages\\nest_asyncio.py'>

In [3]:
import json

with open("fastlane_bot/data/event_test_data.json", "r") as f:
    event_data = json.load(f)

with open("fastlane_bot/data/test_pool_data.json", "r") as f:
    pool_data = json.load(f)

In [4]:

# Create mock instances for all required parameters
cfg = Config.new(config=Config.CONFIG_MAINNET)

# create manager instance for all tests
manager = Manager(web3=cfg.w3,
    w3_async=cfg.w3_async,
    cfg=cfg, 
    pool_data=pool_data, 
    alchemy_max_block_fetch=20, 
    SUPPORTED_EXCHANGES=['bancor_v3', 'carbon_v1', 'uniswap_v2', 'uniswap_v3'])


2023-12-18 11:30:52,478 [fastlane:INFO] - 
2023-12-18 11:30:52,478 [fastlane:INFO] - **********************************************
2023-12-18 11:30:52,478 [fastlane:INFO] - The logging path is set to: logs/20231218-113052\bot.log
2023-12-18 11:30:52,479 [fastlane:INFO] - **********************************************
2023-12-18 11:30:52,479 [fastlane:INFO] - 


## test_update_from_event_carbon_v1_pair_create

In [5]:
event = event_data['carbon_v1_event_pair_created']
assert (event['args']['token0'], event['args']['token1']) not in manager.fee_pairs

manager.update_from_event(event)

assert (event['args']['token0'], event['args']['token1']) in manager.fee_pairs


## test_update_from_event_carbon_v1_trading_fee_updated


In [6]:

event = event_data['carbon_v1_trading_fee_updated']
prevFeePPM = event['args']['prevFeePPM']
newFeePPM = event['args']['newFeePPM']

mocked_contract = Mock()
new_mocked_contract = Mock()
mocked_contract.functions.tradingFeePPM.return_value.call =  AsyncMock(return_value=prevFeePPM)
new_mocked_contract.functions.tradingFeePPM.return_value.call = AsyncMock(return_value=newFeePPM)

@pytest.mark.asyncio
async def test_update_from_event_carbon_v1_trading_fee_updated():
    val = await manager.exchanges['carbon_v1'].get_fee('', mocked_contract)
    assert int(val[0]) == prevFeePPM
    
    # find all pools with fee==prevFeePPM
    prev_default_pools = [idx for idx, pool in enumerate(manager.pool_data) if pool['fee'] == prevFeePPM]
    
    manager.update_from_event(event)

    for idx in prev_default_pools:
        assert manager.pool_data[idx]['fee'] == newFeePPM
    
    val2 = await manager.exchanges['carbon_v1'].get_fee('', new_mocked_contract)
    assert int(val2[0]) == newFeePPM

# Run the test in an event loop
asyncio.run(test_update_from_event_carbon_v1_trading_fee_updated())

## test_update_from_event_carbon_v1_pair_trading_fee_updated

In [7]:
event = event_data['carbon_v1_pair_trading_fee_updated']
prevFeePPM = event['args']['prevFeePPM']
newFeePPM = event['args']['newFeePPM']
token0 = event['args']['token0']
token1 = event['args']['token1']

# set the fee for the pair to prevFeePPM
idxs = [idx for idx, pool in enumerate(manager.pool_data) if pool['tkn0_address'] == token0 and pool['tkn1_address'] == token1 and pool['exchange_name'] == 'carbon_v1']
for idx in idxs:
    manager.pool_data[idx]['fee'] = f"{prevFeePPM}"
    manager.pool_data[idx]['fee_float'] = prevFeePPM / 1e6

# set all other pools with a different fee than prevFeePPM
others = [i for i, pool in enumerate(manager.pool_data) if i not in idxs and pool['exchange_name'] == 'carbon_v1']
for i in others:
    manager.pool_data[i]['fee'] = f"{prevFeePPM-1}"
    manager.pool_data[i]['fee_float'] = (prevFeePPM-1) / 1e6

manager.update_from_event(event)

# check that the fee for the pair is now newFeePPM
for idx in idxs:
    assert manager.pool_data[idx]['fee'] == f"{newFeePPM}"
    assert manager.pool_data[idx]['fee_float'] == newFeePPM / 1e6
    
# check that all other pools have not been changed
for i in others:
    assert manager.pool_data[i]['fee'] == f"{prevFeePPM-1}"
    assert manager.pool_data[i]['fee_float'] == (prevFeePPM-1) / 1e6
